# BIBLIOTECA

In [2]:
pip install certifi


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip-system-certs


Usage:   
  /usr/local/bin/python3 -m pip <command> [options]

no such option: -s
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import ssl
import requests
import certifi
from io import StringIO
import datetime
import time

# WEB SCRAPING BCB ORIGINAL

In [7]:
ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

,id,dataMovimento,dataLiquidacao,edital,tipoPublico,prazo,quantidadeOfertada,quantidadeAceita,codigoTitulo,dataVencimento,tipoOferta,ofertante,quantidadeOfertadaSegundaRodada,quantidadeAceitaSegundaRodada,cotacaoMedia,cotacaoCorte,taxaMedia,taxaCorte,financeiro
0,9fe03e655dd662f9404d14f800000ed,2025-03-13 00:00:00,2025-03-14 00:00:00,55,TodoMercado,383,6000000,6000000,100000,2026-04-01 00:00:00,Venda,Tesouro Nacional,1500000,140000,865784757,865769607,147479,147499,5316
1,9fe03e655dd662f9404d14f800000f2,2025-03-13 00:00:00,2025-03-14 00:00:00,56,TodoMercado,3580,3000000,3000000,950199,2035-01-01 00:00:00,Venda,Tesouro Nacional,750000,749800,779048139,779034654,150271,150275,29216
2,9fe03e655dd662f9404d14f800000f1,2025-03-13 00:00:00,2025-03-14 00:00:00,56,TodoMercado,2119,3000000,3000000,950199,2031-01-01 00:00:00,Venda,Tesouro Nacional,750000,749999,841995679,84175477,149873,14995,31578
3,9fe03e655dd662f9404d14f800000f0,2025-03-13 00:00:00,2025-03-14 00:00:00,55,TodoMercado,2484,3000000,3000000,100000,2032-01-01 00:00:00,Venda,Tesouro Nacional,750000,749999,390056495,389976318,149389,149424,14629
4,9fe03e655dd662f9404d14f800000ef,2025-03-13 00:00:00,2025-03-14 00:00:00,55,TodoMercado,1389,12000000,12000000,100000,2029-01-01 00:00:00,Venda,Tesouro Nacional,3000000,2999999,595948347,595772439,147007,147097,89402


# SELIC - RESULTADOS DOS LEILÕES

In [8]:
import ssl
import pandas as pd
from datetime import datetime

ssl._create_default_https_context = ssl._create_unverified_context

# URL da tabela HTML
url = "https://olinda.bcb.gov.br/olinda/servico/leiloes_selic/versao/v1/odata/leiloesTitulosPublicos(dataMovimentoInicio=@dataMovimentoInicio,dataMovimentoFim=@dataMovimentoFim,dataLiquidacao=@dataLiquidacao,codigoTitulo=@codigoTitulo,dataVencimento=@dataVencimento,edital=@edital,tipoPublico=@tipoPublico,tipoOferta=@tipoOferta)?$top=100&$format=text/html"

# Leitura da tabela HTML usando pandas
tabela = pd.read_html(url)[0]  # [0] para selecionar a primeira tabela da página, se houver várias.

# 1. Reorganizar as colunas
colunas_desejadas = ['edital', 'tipoOferta', 'codigoTitulo', 'ofertante', 'prazo', 'dataLiquidacao','quantidadeOfertada', 'quantidadeAceita', 'taxaCorte']
tabela = tabela[colunas_desejadas]

# 2. Criar a coluna "NomeTitulo"
tabela['NomeTitulo'] = tabela['codigoTitulo'].map({210100: 'LFT 210100', 760199: 'NTN-B 760199'})

# 3. Mover a coluna "NomeTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('codigoTitulo') + 1, colunas.pop(colunas.index('NomeTitulo')))
tabela = tabela[colunas]

# 4. Ordenar por 'edital' com ordem personalizada
ordem_edital = [37, 38, 39]
tabela['edital'] = pd.Categorical(tabela['edital'], categories=ordem_edital, ordered=True)

# 5. Ordenar por 'prazo' com ordem personalizada
ordem_prazo = sorted(tabela['prazo'].unique())
tabela['prazo'] = pd.Categorical(tabela['prazo'], categories=ordem_prazo, ordered=True)

# 6. Criar a coluna "AnoTitulo"
tabela['AnoTitulo'] = (datetime.now() + pd.to_timedelta(pd.to_numeric(tabela['prazo'], errors='coerce'), unit='D')).dt.year

# 7. Mover a coluna "AnoTitulo" para a posição correta
colunas = list(tabela.columns)
colunas.insert(colunas.index('NomeTitulo') + 1, colunas.pop(colunas.index('AnoTitulo')))
tabela = tabela[colunas]

# 8. Formatar a coluna "taxaCorte" como porcentagem
tabela['taxaCorte'] = pd.to_numeric(tabela['taxaCorte'], errors='coerce') / 10000
tabela['taxaCorte'] = tabela['taxaCorte'].astype(str) + '%'

# 9. Formatar a coluna "quantidadeAceita"
tabela['quantidadeAceita'] = tabela['quantidadeAceita'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))


# 10. Formatar a coluna "quantidadeOfertada"
tabela['quantidadeOfertada'] = tabela['quantidadeOfertada'].apply(lambda x: f'{x:,.0f}'.replace(',', '.'))


# Agora você tem a tabela em um DataFrame pandas chamado "tabela"
# Para ver as primeiras linhas da tabela:
tabela.head()

,edital,tipoOferta,codigoTitulo,NomeTitulo,AnoTitulo,ofertante,prazo,dataLiquidacao,quantidadeOfertada,quantidadeAceita,taxaCorte
0,NaN,Venda,100000,NaN,2026,Tesouro Nacional,383,2025-03-14 00:00:00,6.000.000,6.000.000,14.7499%
1,NaN,Venda,950199,NaN,2035,Tesouro Nacional,3580,2025-03-14 00:00:00,3.000.000,3.000.000,15.0275%
2,NaN,Venda,950199,NaN,2031,Tesouro Nacional,2119,2025-03-14 00:00:00,3.000.000,3.000.000,1.4995%
3,NaN,Venda,100000,NaN,2032,Tesouro Nacional,2484,2025-03-14 00:00:00,3.000.000,3.000.000,14.9424%
4,NaN,Venda,100000,NaN,2029,Tesouro Nacional,1389,2025-03-14 00:00:00,12.000.000,12.000.000,14.7097%
